## Use Gemini Vision Model to Convert an ER Diagram into a Graph Model


Version 0.1
Last updated: 2024-02-19

Contact: Fanghua Yu / https://www.linkedin.com/in/joshuayu/



### Install Vertex AI SDK for Python


In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Define Google Cloud project information (Colab only)

If you are running this notebook on Google Colab, specify the Google Cloud project information to use. In the following cell, you specify your project information, import the Vertex AI package, and initialize the package. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [2]:
if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = "YOUR-PROJECT-ID"  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries


In [3]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

## Use the Gemini 1.0 Pro Vision model

Gemini 1.0 Pro Vision (`gemini-1.0-pro-vision`) is a multimodal model that supports multimodal prompts. You can include text, image(s), and video in your prompt requests and get text or code responses.


In [4]:
# Specify the Gemini model to use

multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

### Define helper functions


In [5]:
# The follow code was orginally created as part of the Google Vertex AI sample project.

import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def display_content_as_image(content: str | Image | Part) -> bool:
    if not isinstance(content, Image):
        return False
    display_images([content])
    return True



def print_multimodal_prompt(contents: list[str | Image | Part]):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if display_content_as_image(content):
            continue
        if display_content_as_video(content):
            continue
        print(content)

## Understanding Components in an E-R Diagram


Gemini Vision Pro model has the capabilities to understand diagrams and take actionable steps, such as optimization or code generation. Here we will use the movie rating example to demonstrate how Gemini can decipher an entity relationship (ER) diagram, understand the relationships between tables, and generate schema descriptions in JSON layout.


![E-R Diagram for Movie Rating](https://github.com/Joshua-Yu/graph-rag/raw/main/gemini-multimodal/resources/movie-er.jpg) 



In [6]:

image_er_url = "https://github.com/Joshua-Yu/graph-rag/raw/main/gemini-multimodal/resources/movie-er.jpg"
image_er = load_image_from_url(image_er_url)  

prompt = "Document the entities and relationships in this ER diagram and structure your response in JSON format for entity, relationship and their fields."

contents = [prompt, image_er]

# Use a more deterministic configuration with a low temperature
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    candidate_count=1,
    max_output_tokens=2048,
)

responses = multimodal_model.generate_content(
    contents,
    generation_config=generation_config,
    stream=True,
)

# print("-------Prompt--------")
# print_multimodal_prompt(contents)

finalResponse = ""

print("\n-------Response--------")
for response in responses:
    # Because streaming mode is enabled, we need to collect all pieces of generated text
    finalResponse += response.text
    print(response.text)

    


-------Response--------
 ```json
{
  "entities": [
    {
      "name": "Actor",
      "fields": [
        {

          "name": "ActorID",
          "type": "Integer",
          "primary_key": true,
          "not_null":
 true
        },
        {
          "name": "FirstName",
          "type": "Varchar(50)",
          "not_
null": true
        },
        {
          "name": "LastName",
          "type": "Varchar(30)",
          "
not_null": true
        },
        {
          "name": "Nationality",
          "type": "Varchar(40)",
          "not_null": true
        },
        {
          "name": "BirthDate",
          "type": "Date",
          "
not_null": true
        }
      ]
    },
    {
      "name": "Award",
      "fields": [
        {
          "name": "AwardID",
          "type": "Integer",
          "primary_key": true,
          "not
_null": true
        },
        {
          "name": "AwardName",
          "type": "Varchar(50)",
          "not_null": true
        }
      ]

In [8]:
import json

#finalResponse = finalResponseCache

finalResponse = finalResponse.replace("true", "\"true\"").replace("false", "\"false\"").replace("_","-").replace("```json", "").replace("```","").strip()

responseJson = json.loads(finalResponse)

#print(responseJson)





In [9]:

cypher_constraints = ""
cypher_load_nodes  = ""
cypher_build_relationships = ""

# a.1 Iterate through entities and create UNIQUENESS and/or EXISTENCE constraints
for entity in responseJson.get("entities"): 
    entity_name = entity.get("name")
    
    for field in entity.get("fields"):
        field_name = field.get("name")
        
        if field.get("primary-key") == "true": 
            cypher = f"CREATE CONSTRAINT {entity_name}_{field_name}_unique FOR (n:{entity_name}) REQUIRE n.{field_name} IS UNIQUE;\n"
            cypher_constraints += cypher
            
        if field.get("not-null") == "true" and not field.get("primary-key") == "true": # primary-key must be unique
            cypher = f"CREATE CONSTRAINT {entity_name}_{field_name}_notnull FOR (n:{entity_name}) REQUIRE author.{field_name} IS NOT NULL;\n"
            cypher_constraints += cypher

            
print(cypher_constraints)



    
# TO DO: 
#    - composite key
#    - check and override property value
#    - assume every table has a primary key
#    - entity file, relationship file
    

CREATE CONSTRAINT Actor_ActorID_unique FOR (n:Actor) REQUIRE n.ActorID IS UNIQUE;
CREATE CONSTRAINT Actor_FirstName_notnull FOR (n:Actor) REQUIRE author.FirstName IS NOT NULL;
CREATE CONSTRAINT Actor_LastName_notnull FOR (n:Actor) REQUIRE author.LastName IS NOT NULL;
CREATE CONSTRAINT Actor_Nationality_notnull FOR (n:Actor) REQUIRE author.Nationality IS NOT NULL;
CREATE CONSTRAINT Actor_BirthDate_notnull FOR (n:Actor) REQUIRE author.BirthDate IS NOT NULL;
CREATE CONSTRAINT Award_AwardID_unique FOR (n:Award) REQUIRE n.AwardID IS UNIQUE;
CREATE CONSTRAINT Award_AwardName_notnull FOR (n:Award) REQUIRE author.AwardName IS NOT NULL;
CREATE CONSTRAINT Genre_GenreID_unique FOR (n:Genre) REQUIRE n.GenreID IS UNIQUE;
CREATE CONSTRAINT Genre_GenreName_notnull FOR (n:Genre) REQUIRE author.GenreName IS NOT NULL;
CREATE CONSTRAINT Language_LanguageID_unique FOR (n:Language) REQUIRE n.LanguageID IS UNIQUE;
CREATE CONSTRAINT Language_Language_notnull FOR (n:Language) REQUIRE author.Language IS NOT NU

In [10]:


# a.2 Iterate through entities and generate LOAD CSV statement based on entity schema
#
#     Description: 
#       - load data from a CSV having column headers
#       - bulk commit
#       - using MERGE for UPSERT


for entity in responseJson.get("entities"): 
    entity_name = entity.get("name")
    
    cypher_load_nodes = f"// ----------- LOAD CSV for nodes of {entity_name} -----------\n"
    cypher_load_nodes += f":auto\nWITH $filename\nLOAD CSV FROM $filename AS line\nCALL " + "{" + f"\nMERGE (n:{entity_name} "
    set_statement = "SET "
    
    for field in entity.get("fields"):
        field_name = field.get("name")
    
        if field.get("primary-key") == "true": 
            cypher_load_nodes += "{" + f"{field_name}:line.{field_name}" + "}" + ")\n"
        else: 
            set_statement = set_statement + f"n.{field_name} = line.{field_name},"
    
    cypher_load_nodes += set_statement[:len(set_statement)-1]
    cypher_load_nodes += "\n} IN TRANSACTIONS 2000;"
    
    print(cypher_load_nodes)
    

// ----------- LOAD CSV for nodes of Actor -----------
:auto
WITH $filename
LOAD CSV FROM $filename AS line
CALL {
MERGE (n:Actor {ActorID:line.ActorID})
SET n.FirstName = line.FirstName,n.LastName = line.LastName,n.Nationality = line.Nationality,n.BirthDate = line.BirthDate
} IN TRANSACTIONS 2000;
// ----------- LOAD CSV for nodes of Award -----------
:auto
WITH $filename
LOAD CSV FROM $filename AS line
CALL {
MERGE (n:Award {AwardID:line.AwardID})
SET n.AwardName = line.AwardName
} IN TRANSACTIONS 2000;
// ----------- LOAD CSV for nodes of Genre -----------
:auto
WITH $filename
LOAD CSV FROM $filename AS line
CALL {
MERGE (n:Genre {GenreID:line.GenreID})
SET n.GenreName = line.GenreName
} IN TRANSACTIONS 2000;
// ----------- LOAD CSV for nodes of Language -----------
:auto
WITH $filename
LOAD CSV FROM $filename AS line
CALL {
MERGE (n:Language {LanguageID:line.LanguageID})
SET n.Language = line.Language
} IN TRANSACTIONS 2000;
// ----------- LOAD CSV for nodes of Movie -----------
:a

In [11]:

# a.3 Iterate Relationships to create relationship between nodes
#
# Naming conventions: 
#   - for source_node:SourceLabel -> target_node:TargetLabel, the name of relationship is HAS_<TargetLabel> in big cases

for relationship in responseJson.get("relationships"):
    source_entity = relationship.get("source-entity")
    source_field  = relationship.get("source-field")
    target_entity = relationship.get("target-entity")
    target_field  = relationship.get("target-field")
    relation_name = relationship.get("name")
    
    
    cypher_create_relationships = f"// ----------- LOAD CSV for relationships of {relation_name} -----------\n"
    cypher_create_relationships += f":auto\nWITH $filename\nLOAD CSV FROM $filename AS line\nCALL " + "{\n"
    cypher_create_relationships += f"MATCH (e1:{source_entity}" + "{" + f"{source_field}:line.{source_field}" + "})\n"
    cypher_create_relationships += f"MATCH (e2:{target_entity}" + "{" + f"{target_field}:line.{target_field}" + "})\n"
    cypher_create_relationships += f"MERGE (e1) -[:HAS_{target_entity.upper()}]-> (e2)\n" + "} IN TRANSACTIONS 2000;\n"
    
    print(cypher_create_relationships)



// ----------- LOAD CSV for relationships of Actor-Award -----------
:auto
WITH $filename
LOAD CSV FROM $filename AS line
CALL {
MATCH (e1:Actor{ActorID:line.ActorID})
MATCH (e2:Award{AwardID:line.AwardID})
MERGE (e1) -[:HAS_AWARD]-> (e2)
} IN TRANSACTIONS 2000;

// ----------- LOAD CSV for relationships of Movie-Genre -----------
:auto
WITH $filename
LOAD CSV FROM $filename AS line
CALL {
MATCH (e1:Movie{GenreID:line.GenreID})
MATCH (e2:Genre{GenreID:line.GenreID})
MERGE (e1) -[:HAS_GENRE]-> (e2)
} IN TRANSACTIONS 2000;

// ----------- LOAD CSV for relationships of Movie-Language -----------
:auto
WITH $filename
LOAD CSV FROM $filename AS line
CALL {
MATCH (e1:Movie{LanguageID:line.LanguageID})
MATCH (e2:Language{LanguageID:line.LanguageID})
MERGE (e1) -[:HAS_LANGUAGE]-> (e2)
} IN TRANSACTIONS 2000;

// ----------- LOAD CSV for relationships of Movie-Director -----------
:auto
WITH $filename
LOAD CSV FROM $filename AS line
CALL {
MATCH (e1:Movie{DirectorID:line.DirectorID})
MATCH (e2: